In [1]:
import pandas as pd
from utils.openai_query import openai_chat
import re
import json

In [2]:
with open('jsonFiles/OmicsRunLLM.json', 'r') as f:
    config = json.load(f)
context = config['CONTEXT']
max_tokens = 2000
rate_per_token = config['RATE_PER_TOKEN']
LOG_FILE = config['LOG_NAME']
DOLLAR_LIMIT = config['DOLLAR_LIMIT']


In [4]:
# Read the results of processing the 300 'omics datasets
temp = pd.read_csv("data/omics_revamped_LLM_updated_genes_DF.tsv", delimiter="\t")
input_analyses = temp[temp["LLM Name"] != "System of unrelated proteins"]
print(len(temp[temp['Score']>0]))
print(len(input_analyses))
input_analyses.head(3)

135
135


,Source,GeneSetID,GeneSetName,GeneList,updated GeneList,n_Genes,LLM Name,LLM Analysis,Score
0,NeST,Cluster1-10,Cluster1-10,CTRL HSD17B14 KIAA0232 PAQR8 PLA2G1B RNF145 SG...,CTRL HSD17B14 KIAA0232 PAQR8 PLA2G1B RNF145 SG...,12,Lipid Metabolism and Membrane Dynamics,"1. CTRL, or carboxyl-terminal esterase/lipase,...",0.85
1,NeST,Cluster1-11,Cluster1-11,LMF1 MFHAS1 MR1 PLA2G1B RASL11A RNF145 SLC2A6 ...,LMF1 MFHAS1 MR1 PLA2G1B RASL11A RNF145 SLC2A6 ...,12,Lipid Metabolism and Membrane Trafficking,1. LMF1 (Lipase Maturation Factor 1) is crucia...,0.85
2,NeST,Cluster1-12,Cluster1-12,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,12,Lipid Metabolism and Receptor-Mediated Endocyt...,"1. AMY2B (Amylase, alpha 2B) is an enzyme that...",0.88


In [30]:
count_genes_prompt_template_v1 = """

The provided text is an analysis of a gene set to describe its common functions and to provide a name for the gene set reflecting the predominant function or functions.

Do not critique the analysis or the name. Your job is to find the genes that support the name

You are provided with this list of the gene symbols mentioned in the text. Only consider these genes.

<genes mentioned in the text>{genes_in_text}</genes mentioned in the text>
 
For each gene from that list mentioned in a sentence or paragraph, decide whether the text is making a *definite* assertion about the gene that supports the name.
DO NOT make your own assesments about the gene's function or the validity of the analysis, just evaluate the text.
<example of definite assertion>
"XRCC1 is involved in the DNA damage response"
</example of definite assertion>
<example of non-definite assertion>
"E2F1 may be involved in homolougus recombination."
"SREBF1 could be related to RAD51's function."
</example of non-definite assertion>

As you work, briefly explain your reasoning for each gene symbol that supports the name.
<example of explanation>
Reasoning:

1. CITED2: Involved in transcriptional regulation influencing gene expression, which is crucial for tissue development.
2. TWIST1: Crucial for mesodermal tissue development and epithelial-mesenchymal transition (EMT), directly supporting tissue development.
3. LMO2: Implicated in hematopoiesis and vascular development, supporting tissue development.
4. COL5A1: Encodes a component of type V collagen, essential for connective tissue structure and function, supporting tissue development and extracellular matrix remodeling.
</example of explanation>

For each gene symbol that supports the name, add it to your list of supporting genes.

<text>{text}</text>

<name>{name}</name>

Output the supporting genes and non-supporting genes in the following format: 

<format>
-- Explanation --
<explanation of the reasoning for the supporting genes>
-- genes supporting the name: <list of gene symbols of genes supporting the name>
</format>
"""

count_genes_prompt_template_v2 = """
Analyze the provided text, which describes a gene set's common functions and suggests a name reflecting its predominant function(s). Your task is to identify genes that support this name based solely on the information given in the text.

Context: Gene sets are groups of genes that share common biological functions, pathways, or other characteristics. Naming these sets based on their predominant functions helps researchers quickly understand their significance.

Input:
1. A list of gene symbols in the gene set, provided in comma-separated format:
<gene set>{genes_in_text}</gene set>

2. The analysis text:
<text>{text}</text>

3. The suggested name for the gene set:
<name>{name}</name>

Instructions:
1. Evaluate each gene from the provided list that is mentioned in the text.
2. 2. Consider a gene as supporting the suggested name if (and only if) it makes a definite assertion about the gene in support of the supporting name.
   - A definite assertion clearly states a gene's function or role without using speculative language.
   - Example of a definite assertion: "XRCC1 is involved in the DNA damage response"
   - Example of a non-definite assertion: "E2F1 may be involved in homologous recombination"
3. If a gene is mentioned multiple times, consider the strongest assertion made about it.
5. For each gene you determine supports the name:
   - Briefly explain your reasoning (max 50 words per gene)
   - Assign a confidence level (High, Medium, Low) based on the strength of the assertion
6. Handle acronyms or alternative gene names as equivalent to official gene symbols.
7. If no genes seem to support the name or if all genes support it, state this observation.

Output your analysis in the following format:

-- Summary --
[Provide a brief summary (max 100 words) of why the selected genes support the given name]

-- Explanation --
[Gene Symbol]: [Confidence Level]
[Explanation of reasoning (max 50 words)]

[Repeat for each supporting gene]

-- genes supporting the name: [List of gene symbols of genes supporting the name]
-- genes supporting the name with confidence = High: [List of gene symbols with high confidence]

Do not critique the analysis or the name. Base your evaluation solely on the information provided in the text.
"""

count_genes_prompt_template = """
Analyze the provided text, which describes a gene set's common functions and suggests a name reflecting its predominant function(s). Your task is to identify genes that support this name based solely on the information given in the text.

Context: Gene sets are groups of genes that share common biological functions, pathways, or other characteristics. Naming these sets based on their predominant functions helps researchers quickly understand their significance.

Input:
1. A list of gene symbols in the gene set, provided in comma-separated format:
<gene set>{genes_in_text}</gene set>

2. The analysis text:
<text>{text}</text>

3. The suggested name for the gene set:
<name>{name}</name>

Instructions:
1. Evaluate each gene from the provided list that is mentioned in the text.
2. Determine if the text makes a definite assertion about the gene that supports the given name.
   - A definite assertion clearly states a gene's function or role without using speculative language.
   - Example of a definite assertion: "XRCC1 is involved in the DNA damage response"
   - Example of a non-definite assertion: "E2F1 may be involved in homologous recombination"
3. If a gene is mentioned multiple times, consider the strongest assertion made about it.
4. In case of contradictory statements about a gene, favor the most recent or specific assertion.
5. For each gene you determine supports the name:
   - Briefly explain your reasoning (max 50 words per gene)
   - Assign a confidence level (High, Medium, Low) based on the strength of the assertion
6. Handle acronyms or alternative gene names as equivalent to official gene symbols.
7. If no genes seem to support the name or if all genes support it, state this observation.

Output your analysis in the following format:

-- Summary --
[Provide a brief summary (max 100 words) of why the selected genes support the given name]

-- Explanation --
[Gene Symbol]: [Confidence Level]
[Explanation of reasoning (max 50 words)]

[Repeat for each supporting gene]

-- genes supporting the name: [List of gene symbols of genes supporting the name]
-- genes supporting the name with confidence = High: [List of gene symbols with high confidence]

Do not critique the analysis or the name. Base your evaluation solely on the information provided in the text.
"""

def parse_analysis(text):
    formatted_genes = ""
    supporting_count = 0
    try:
        #supporting_genes = re.search(r"genes supporting the name: (.+)", text).group(1).strip("[]").split(", ")
        supporting_genes = re.search(r"genes supporting the name with confidence = High: (.+)", text).group(1).strip("[]").split(", ")
        supporting_count = len(supporting_genes)
        formatted_genes = " ".join(supporting_genes)
    except Exception as e:
        print(text)

    return formatted_genes, supporting_count

def get_genecount_mentioned_in_text(genelist, text):
    genes = set(genelist.split(" "))
    number_mentioned = sum(1 for gene in genes if gene in text)
    return number_mentioned

def get_genes_supporting_name(name, text, prompt_template, genes_in_text):
    prompt = prompt_template.format(genes_in_text=genes_in_text, text=text, name=name)
    temperature = 0.0
    model = "gpt-4o"
    analysis, finger_print  = openai_chat(context, prompt, model, temperature, max_tokens, rate_per_token, LOG_FILE, DOLLAR_LIMIT)
    # print(analysis)
    supporting_genes, supporting_count = parse_analysis(analysis)
    return supporting_genes, supporting_count, analysis

def process_gene_set_analyses(input_analyses, prompt_template, number_to_process=None, progress_file="data/supporting_gene_counts_progress.tsv"):
    if number_to_process is not None:
        analyses = input_analyses.head(number_to_process).copy()
    else:
        analyses = input_analyses.copy()
    
    print(f"Processing {len(analyses)} analyses")

    def get_mentioned(row): 
        return get_genecount_mentioned_in_text(row["GeneList"], row["LLM Analysis"])

    def get_supporting(row):
        print(f'processing {row["GeneSetName"]}...')
        return get_genes_supporting_name(row["LLM Name"], row["LLM Analysis"], prompt_template, row["GeneList"])
    
    #initialize columns
    analyses["Supporting Genes"] = None
    analyses["Supporting Count"] = None
    analyses["LLM Support Analysis"] = None
    analyses["GenesMentionedInText"] = None

    for idx, row in analyses.iterrows():
        supporting_genes, supporting_count, supporting_analysis = get_supporting(row)
        mentioned_genes = get_mentioned(row)
        analyses.loc[idx, "Supporting Genes"] = supporting_genes
        analyses.loc[idx, "Supporting Count"] = supporting_count
        analyses.loc[idx, "LLM Support Analysis"] = supporting_analysis
        analyses.loc[idx, "GenesMentionedInText"] = mentioned_genes

        # results = analyses[[
        #     "Source", "GeneSetID", "GeneSetName", "GeneList", "n_Genes", "Supporting Genes", 
        #     "Supporting Count", "LLM Analysis",
        #     "LLM Support Analysis", "GenesMentionedInText"]].copy()
        analyses.to_csv(progress_file, sep='\t', index=False)
    
    return analyses

In [ ]:
results_df = process_gene_set_analyses(input_analyses, count_genes_prompt_template, number_to_process=None)
results_df.loc[:,'LLM_coverage'] = results_df.apply(lambda row: round(row['Supporting Count'] / row['n_Genes'], 2), axis=1)
# results_df.to_csv("data/omics_revamped_LLM_Enrichr_simVals_new_genecounts_unprocessed__DF.tsv", sep="\t", index=False)
# results_df.head()

In [ ]:
processed_results_df = results_df.copy()

# Replace newlines within text values with a placeholder
processed_results_df = processed_results_df.map(lambda x: str(x).replace('\n', '||'))

processed_results_df.to_csv("data/omics_revamped_LLM_genecounts_DF.tsv", sep="\t", index=False)
processed_results_df.head()